In [460]:
import pandas as pd
import json
import re
import numpy as np

In [461]:
# load the dataset
json_file = open('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_dataset.json', 'r')
#, encoding='utf-8'
data = json.load(json_file)

In [462]:
# Extract the variables of the first page
name = data[0]['name'].copy()
links = data[0]['link'].copy()
birth_year = data[0]['birth_year'].copy()
birth_place = data[0]['birth_place'].copy()
death_year = data[0]['death_year'].copy()
death_place = data[0]['death_place'].copy()

In [463]:
# Get the full links of the second pages before creating the dataframe
full_links = []
for link in links:
    full = 'http://www.ifar.org/' + link
    full_links.append(full)

In [464]:
# Check that the variables was created successfully
full_links[:5]

['http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=1397&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=3073&published=1&inPrep=1&author=',
 'http://www.ifar.org/artist_name.php?nameid=2030&published=1&inPrep=1&author=']

In [465]:
len(full_links)

2937

In [466]:
# Create the first dataset containing all of page 1
page_one = pd.DataFrame(list(zip(name, full_links, birth_year, birth_place, death_year, death_place)),
                          columns = ['names', 'links', 'birth_year', 'birth_place', 'death_year', 'death_place'])

In [467]:
page_one.head()

,names,links,birth_year,birth_place,death_year,death_place
0,"Aachen, Hans von",http://www.ifar.org/artist_name.php?nameid=156...,"<td class=""g_center_birth_year"">1552</td>","<td class=""g_center_birth_place"">Germany</td>","<td class=""g_center_death_year"">1615</td>","<td class=""g_center_death_place"">Czech Republi..."
1,"Aalto, Alvar",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1898</td>","<td class=""g_center_birth_place"">Finland</td>","<td class=""g_center_death_year"">1976</td>","<td class=""g_center_death_place"">Finland</td>"
2,"Abbati, Giuseppe",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1836</td>","<td class=""g_center_birth_place"">Italy</td>","<td class=""g_center_death_year"">1868</td>","<td class=""g_center_death_place"">Italy</td>"
3,"Abdessemed, Adel",http://www.ifar.org/artist_name.php?nameid=307...,"<td class=""g_center_birth_year"">1971</td>","<td class=""g_center_birth_place"">Algeria</td>","<td class=""g_center_death_year""></td>","<td class=""g_center_death_place""></td>"
4,"Abelenda Zapata, Manuel",http://www.ifar.org/artist_name.php?nameid=203...,"<td class=""g_center_birth_year"">1889</td>","<td class=""g_center_birth_place"">Spain</td>","<td class=""g_center_death_year"">1957</td>","<td class=""g_center_death_place"">Spain</td>"


In [468]:
# Clean all of the variables in the first page and add them again to the dataset
regex = r'^(\d{4})'
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

clean_byear = []
clean_bplace = []
clean_dyear = []
clean_dplace = []

for by in page_one['birth_year']:
    f = remove_html_tags(by)
    clean_byear.append(f)
    
for bp in page_one['birth_place']:
    q = remove_html_tags(bp)
    clean_bplace.append(q)
    
for dy in page_one['death_year']:
    e = remove_html_tags(dy)
    clean_dyear.append(e)
    
for dp in page_one['death_place']:
    w = remove_html_tags(dp)
    clean_dplace.append(w)

page_one['birth_year'] = pd.Series(clean_byear)
page_one['birth_place'] = pd.Series(clean_bplace)
page_one['death_year'] = pd.Series(clean_dyear)
page_one['death_place'] = pd.Series(clean_dplace)

In [469]:
# Make sure that the variables were cleaned correctly
page_one.tail()

,names,links,birth_year,birth_place,death_year,death_place
2932,"Zuccaro, Taddeo",http://www.ifar.org/artist_name.php?nameid=275...,1529,Italy,1566,Italy
2933,"Zuloaga, Ignacio",http://www.ifar.org/artist_name.php?nameid=273...,1870,Spain,1945,Spain
2934,"Zumthor, Peter",http://www.ifar.org/artist_name.php?nameid=269...,1943,Switzerland,,
2935,"Zuñiga, Francisco",http://www.ifar.org/artist_name.php?nameid=109...,1912,Costa Rica,1998,Mexico
2936,"Zurbarán, Francisco de",http://www.ifar.org/artist_name.php?nameid=110...,1598,Spain,1664,Spain


In [470]:
# Make a copy of the rest of the dataset
second_part = data[1:].copy()

In [471]:
len(second_part)

7926

In [472]:
# Clean all of the variables for the second and third pages
artist_one = []
box_info = []
artist_two = []
column_names = []
content = []

for s in second_part:
    for key, val in s.items():
        if key == 'artist_one':
            artist_one.append(val)
        elif key == 'box_info':
            box_info.append(val)
        elif key == 'artist_two':
            artist_two.append(val)
        elif key == 'column_names':
            column_names.append(val)
        elif key == 'content':
            content.append(val)

In [477]:
len(content), len(name)

(4989, 2937)

In [478]:
# Clean the box
box_new = []
for i in box_info:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    join_them = list(filter(None, tempora))
    box_new.append(join_them)

box_new[0], len(box_new)

(['Published Catalogues',
  'Hans von Aachen, 1552-1615.',
  'click to learn more',
  'Jacoby, Joachim W.',
  'Munich ; Berlin : Deutscher Kunstverlag, 2000.'],
 2937)

In [479]:
# Clean the new artist variable
artist_three = []
for i in artist_two:
    gen = str(i).strip('Artist: ')
    artist_three.append(gen)
    
artist_three[1], len(artist_three)

('Rho, Manlio', 4989)

In [480]:
# Clean the new columns
cols_news = []
for i in column_names:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    cols_news.append(tempora)
    
cols_news[1], len(cols_news)

(['Item Title:',
  'Author:',
  'ISBN:',
  'Imprint:',
  'Language:',
  'Size:',
  'Pages:',
  'Illustrations:',
  'Concordance:',
  'Bibliography:',
  'Index:',
  'Exhibition List:',
  'Chronology:',
  'Content Note:',
  'Individual Entries Contain:',
  'Public Note:'],
 4989)

In [481]:
new_content = []
for i in content:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    new_content.append(tempora)
    
new_content[1], len(new_content)

(['Rho : catalogo generale.',
  'Caramel, Luciano',
  '8843533339',
  'Milan : Electa, 1990.',
  'Italian',
  '29 cm',
  '316',
  'b&w, some color',
  'No',
  'Yes',
  'No',
  'Yes',
  'No',
  'The catalogue is based on the Rho Archives in Como. It is divided into three parts: the chronological catalogue of the paintings, followed by separate catalogues covering drawings for textile designs and decorative projects. \n\nEntries provide title, date (if attributed, date is determined by style, exhibition information, etc), dimensions, written documentation found on the recto or verso. The catalogue also cites current owners, auction sale information, provenance, exhibition and bibliographic information, technical notes, and comments.',
  'Provenance; Exhib. Hist; Bibliog.; Comments',
  'After the introduction, there is a catalogue raisonné, followed by sections dealing with textile designs, graphic designs, and projects for interiors, exhibition displays, and architecture. \n\nThe appendi

In [ ]:
combined_it_cont = []

for item, field in cols_news, new_content:
    temporary = []
    for s, u in item, field:
        val = list(s, u)
        temporary.append(val)
    combined_it_cont.append(temporary)

combined_it_cont[1]

In [493]:
t1 = cols_news.copy()
t2 = new_content.copy()

In [530]:
data_cols = ['Artist name', 'Item Title', 'Status (Published or in Preparation)', 'Author', 'ISBN', 'Imprint',
             'Language', 'Size', 'Pages', 'Illustrations', 'Concordance', 'Bibliography', 'Index',
             'Exhibition List', 'Chronology', 'Individual Entries Contain', 'Public Note']


In [521]:
third_page = pd.DataFrame(list(zip(artist_three, cols_news, new_content)), columns = ['name', 'col_names', 'content'])

In [531]:
for i in data_cols:
    third_page[i] = np.nan

In [532]:
third_page.head()

,name,col_names,content,Artist name,Item Title,Status (Published or in Preparation),Author,ISBN,Imprint,Language,Size,Pages,Illustrations,Concordance,Bibliography,Index,Exhibition List,Chronology,Individual Entries Contain,Public Note
0,"Rilke-Westhoff, Clara","[Item Title:, Author:, ISBN:, Imprint:, Langua...","[Die Bildhauerin Clara Rilke-Westhoff, 1878-19...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Rho, Manlio","[Item Title:, Author:, ISBN:, Imprint:, Langua...","[Rho : catalogo generale., Caramel, Luciano, 8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Ribalta, Francisco","[Item Title:, Author:, ISBN:, Imprint:, Langua...",[Ribalta y los ribaltescos : la evolución del ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Réquichot, Bernard","[Item Title:, Author(s):, Imprint:, Language:,...","[Bernard Réquichot., Barthes, Roland ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Révoil, Pierre-Hen","[Item Title:, Author:, Imprint:, Language:, Si...",[Fleury Richard et Pierre Révoil : la peinture...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in third_page:
    if third_page.col

In [594]:
# Second dataset
page_two = pd.DataFrame(list(zip(artist_one, box_new)), columns = ['name', 'box'])

In [595]:
page_two.head(27)

,name,box
0,"Aachen, Hans von","[Published Catalogues, Hans von Aachen, 1552-1..."
1,"Acht, René Charles","[Published Catalogues, René Acht : Monographie..."
2,"Adam, Otto","[Published Catalogues, Der Maler Otto Adam (19..."
3,"Adams, Clinton","[Published Catalogues, A Meticulous Serenity :..."
4,"Accardi, Carla","[Published Catalogues, Carla Accardi : la vita..."
5,"Adams, Charles Partridge","[Catalogues in Preparation, Title unknown at t..."
6,"Adam, Robert","[Published Catalogues, The Complete Works of R..."
7,"Adam, James","[Published Catalogues, The Complete Works of R..."
8,"Abelenda Zapata, Manuel","[Published Catalogues, Manuel Abelenda : (1889..."
9,"Abdessemed, Adel","[Published Catalogues, Adel Abdessemed : Works..."


In [596]:
occurr_publi = []
for i in page_two.box:
    if 'Published Catalogues' in i:
        occurr_publi.append(1)
    else:
        occurr_publi.append(0)

occurr_publi[1:5]  

[1, 1, 1, 1]

In [597]:
occurr_cat = []
for i in page_two.box:
    if 'Catalogues in Preparation' in i:
        occurr_cat.append(1)
    else:
        occurr_cat.append(0)

occurr_cat[1:5]

[0, 0, 0, 0]

In [600]:
page_two['publi_cat'] = pd.Series(occurr_publi)
page_two['cat_prep'] = pd.Series(occurr_cat)
page_two.head()

,name,box,publi_cat,cat_prep
0,"Aachen, Hans von","[Published Catalogues, Hans von Aachen, 1552-1...",1,0
1,"Acht, René Charles","[Published Catalogues, René Acht : Monographie...",1,0
2,"Adam, Otto","[Published Catalogues, Der Maler Otto Adam (19...",1,0
3,"Adams, Clinton","[Published Catalogues, A Meticulous Serenity :...",1,0
4,"Accardi, Carla","[Published Catalogues, Carla Accardi : la vita...",1,0


In [557]:
tags = page_two['box'].apply(pd.Series)

In [558]:
tags = tags.rename(columns = lambda x : 'tag_' + str(x))

In [560]:
pd.set_option('display.max_columns', 500)
len(tags)

2937